In [6]:
pip install pandas pymysql


  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl (11.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import pymysql

# Função para conectar ao banco de dados MySQL
def connect_to_mysql(host, user, password, database):
    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        return connection
    except pymysql.MySQLError as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

# Configurações de conexão
host = 'localhost'
user = 'root'
password = '123456'
database = 'veterinaria'

# Conectando ao banco de dados
connection = connect_to_mysql(host, user, password, database)


In [12]:
# Função para executar consultas SQL e retornar DataFrames
def execute_query(query, connection):
    try:
        df = pd.read_sql(query, connection)
        return df
    except Exception as e:
        print(f"Erro ao executar a consulta: {e}")
        return None


In [15]:
#Consulta 1: Junção múltipla
query1 = """
SELECT 
    t1.Funcionario_cpf, 
    t2.nome AS Nome_Funcionario,
    t1.Cliente_cpf, 
    t3.cpf AS Nome_Cliente,
    t4.forma_pagamento, 
    t1.Data_hora,
    t6.descricao, 
    t6.quantidade, 
    t6.preco
FROM veterinaria.Venda t1
LEFT JOIN veterinaria.funcionario t2 ON
    t1.Funcionario_cpf = t2.cpf
LEFT JOIN veterinaria.Cliente t3 ON
    t1.Cliente_cpf = t3.cpf
LEFT JOIN veterinaria.pagamento t4 ON
    t1.pagamento_id = t4.id
LEFT JOIN veterinaria.itens_venda t5 ON 
    t1.id = t5.Venda_idVenda
LEFT JOIN veterinaria.Produto t6 ON
    t5.Produto_id = t6.id;
"""

df1 = execute_query(query1, connection)
print(df1)


   Funcionario_cpf Nome_Funcionario  Cliente_cpf Nome_Cliente  \
0        123456789       João Silva  12057103021  12057103021   
1        987654321     Maria Santos  12057193021  12057193021   
2        456789123   Pedro Oliveira  14719263038  14719263038   
3        789123456      Ana Pereira  18315064088  18315064088   

     forma_pagamento           Data_hora     descricao  quantidade   preco  
0  Cartão de Crédito 2023-05-19 10:30:00  Ração Canina        50.0  100.00  
1   Cartão de Débito 2023-05-19 11:00:00          None         NaN    None  
2           Dinheiro 2023-05-19 11:30:00  Ração Felina        30.0   80.00  
3                Pix 2023-05-19 12:00:00          None         NaN    None  


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_17788\4227175894.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [16]:
#Consulta 2: GROUP BY
query2 = """
SELECT 
    t2.nome AS Nome_Funcionario,
    t4.forma_pagamento,  
    SUM(t6.quantidade) AS Qdte_Total, 
    (SUM(t6.quantidade) * SUM(t6.preco)) AS Valor_Total
FROM veterinaria.Venda t1
LEFT JOIN veterinaria.funcionario t2 ON
    t1.Funcionario_cpf = t2.cpf
LEFT JOIN veterinaria.Cliente t3 ON
    t1.Cliente_cpf = t3.cpf
LEFT JOIN veterinaria.pagamento t4 ON
    t1.pagamento_id = t4.id
LEFT JOIN veterinaria.itens_venda t5 ON 
    t1.id = t5.Venda_idVenda
LEFT JOIN veterinaria.Produto t6 ON
    t5.Produto_id = t6.id
GROUP BY
    t1.Funcionario_cpf, 
    t4.forma_pagamento;
"""

df2 = execute_query(query2, connection)
print(df2.head())


  Nome_Funcionario    forma_pagamento  Qdte_Total  Valor_Total
0       João Silva  Cartão de Crédito        50.0       5000.0
1     Maria Santos   Cartão de Débito         NaN          NaN
2   Pedro Oliveira           Dinheiro        30.0       2400.0
3      Ana Pereira                Pix         NaN          NaN


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_17788\4227175894.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [17]:
#Consulta 3: ORDER BY
query3 = """
SELECT 
    t1.Cliente_cpf, 
    t3.cpf AS Nome_Cliente, 
    SUM(t6.quantidade) AS Qdte_Total, 
    (SUM(t6.quantidade) * SUM(t6.preco)) AS Valor_Total
FROM veterinaria.Venda t1
LEFT JOIN veterinaria.funcionario t2 ON
    t1.Funcionario_cpf = t2.cpf
LEFT JOIN veterinaria.Cliente t3 ON
    t1.Cliente_cpf = t3.cpf
LEFT JOIN veterinaria.pagamento t4 ON
    t1.pagamento_id = t4.id
LEFT JOIN veterinaria.itens_venda t5 ON 
    t1.id = t5.Venda_idVenda
LEFT JOIN veterinaria.Produto t6 ON
    t5.Produto_id = t6.id
GROUP BY 
    t1.Cliente_cpf, 
    t3.cpf
ORDER BY 
    Valor_Total DESC;
"""

df3 = execute_query(query3, connection)
print(df3.head())


   Cliente_cpf Nome_Cliente  Qdte_Total  Valor_Total
0  12057103021  12057103021        50.0       5000.0
1  14719263038  14719263038        30.0       2400.0
2  12057193021  12057193021         NaN          NaN
3  18315064088  18315064088         NaN          NaN


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_17788\4227175894.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [18]:
#Consulta 4: LIKE
query4 = """
SELECT 
    t1.nome AS Nome_Animal, 
    t1.especie, 
    t1.idade, 
    t1.peso, 
    t2.nome AS Nome_Cliente
FROM veterinaria.Animal t1
LEFT JOIN veterinaria.Cliente t2 ON
    t1.Cliente_cpf = t2.cpf
WHERE -- Filtrar nome de animal que termine com a letra "o"
    t1.nome LIKE '%o';
"""

df4 = execute_query(query4, connection)
print(df4.head())


  Nome_Animal   especie  idade  peso Nome_Cliente
0       Apolo  Cachorro      7  30.0      Irloaim
1        Nemo     Peixe      2   0.2       Borlia
2       Rocco  Cachorro      6  22.6        Elion
3        Milo      Gato      5   4.9      Farlion
4        Coco     Arara      4   1.5      Geraion


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_17788\4227175894.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [19]:
#Consulta 5: Subconsulta EXISTS
# Consulta 1: Selecionar funcionários que tenham realizado vendas
query5 = """
SELECT nome
FROM veterinaria.funcionario
WHERE EXISTS (
    SELECT 1
    FROM veterinaria.Venda
    WHERE Venda.Funcionario_cpf = funcionario.cpf
);
"""

df5 = execute_query(query5, connection)
print(df5.head())

# Consulta 2: Selecionar produtos que nunca foram vendidos
query6 = """
SELECT id, descricao
FROM veterinaria.Produto AS P
WHERE NOT EXISTS (
    SELECT 1
    FROM veterinaria.itens_venda AS IV
    WHERE IV.Produto_id = P.id
);
"""

df6 = execute_query(query6, connection)
print(df6.head())


             nome
0      João Silva
1  Pedro Oliveira
2     Ana Pereira
3    Maria Santos
   id              descricao
0   3          Medicamento A
1   4    Brinquedo para Cães
2   5     Coleira Antipulgas
3   6      Shampoo para Cães
4   7  Arranhador para Gatos


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_17788\4227175894.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_17788\4227175894.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [20]:
#Consulta 6: LEFT JOIN
query7 = """
SELECT V.id, V.Funcionario_cpf, V.Cliente_cpf, V.pagamento_id, V.Data_hora, IV.`servico-produto_codigo`, IV.Produto_id
FROM veterinaria.Venda V
LEFT JOIN veterinaria.itens_venda IV ON V.id = IV.Venda_idVenda;
"""

df7 = execute_query(query7, connection)
print(df7.head())


   id  Funcionario_cpf  Cliente_cpf  pagamento_id           Data_hora  \
0   1        123456789  12057103021             1 2023-05-19 10:30:00   
1   2        987654321  12057193021             2 2023-05-19 11:00:00   
2   3        456789123  14719263038             3 2023-05-19 11:30:00   
3   4        789123456  18315064088             4 2023-05-19 12:00:00   

   servico-produto_codigo  Produto_id  
0                     NaN         1.0  
1                     1.0         NaN  
2                     NaN         2.0  
3                     2.0         NaN  


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_17788\4227175894.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [21]:
#Consulta 7: Funções de agregação
query8 = """
SELECT C.cpf AS Cliente_cpf, 
       C.nome AS Nome_Cliente,
       COUNT(*) AS Total_Vendas
FROM veterinaria.Venda V
JOIN veterinaria.Cliente C ON V.Cliente_cpf = C.cpf
GROUP BY Cliente_cpf, Nome_Cliente;
"""

df8 = execute_query(query8, connection)
print(df8.head())


   Cliente_cpf Nome_Cliente  Total_Vendas
0  12057103021       Bumbwe             1
1  12057193021       Forond             1
2  14719263038       Reaion             1
3  18315064088        Geurt             1


C:\Users\WTINFO PC\AppData\Local\Temp\ipykernel_17788\4227175894.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [22]:
# Fechando a conexão
if connection:
    connection.close()
